In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib
from dotenv import load_dotenv


ENV_PATH = "/Users/rifat-mac/Study/Data-Analytics-Project/Oslo-city-bike-analytics/.env"
load_dotenv(ENV_PATH)

SERVER  = os.getenv("MSSQL_SERVER", "localhost,1433")
DB_NAME = os.getenv("MSSQL_DB", "OsloBike")  
USER    = os.getenv("MSSQL_UID", "SA")
PWD     = os.getenv("MSSQL_PWD")

if PWD is None:
    raise ValueError("MSSQL_PWD is not set in .env. Please add it.")



DRIVER_PATH = "/opt/homebrew/lib/libmsodbcsql.18.dylib"
if not os.path.exists(DRIVER_PATH):
    raise FileNotFoundError("ODBC driver not found at /opt/homebrew/lib/libmsodbcsql.18.dylib")

os.environ["PATH"] += ":/opt/homebrew/bin:/opt/homebrew/lib"

conn_str = (
    f"DRIVER={DRIVER_PATH};"
    f"SERVER={SERVER};"
    f"DATABASE={DB_NAME};"
    f"UID={USER};PWD={PWD};"
    "TrustServerCertificate=yes;"
    "Encrypt=no;"
)

params = urllib.parse.quote_plus(conn_str)
engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={params}",
    fast_executemany=True
)


BASE = "/Users/rifat-mac/Study/Data-Analytics-Project/Oslo-city-bike-analytics/data/processed"

TRIPS_CSV    = os.path.join(BASE, "Trips_SQL_2025_05_10.csv")
STATIONS_CSV = os.path.join(BASE, "Stations_WithArea_SQL.csv")

print("Trips CSV   :", TRIPS_CSV)
print("Stations CSV:", STATIONS_CSV)


df_st = pd.read_csv(STATIONS_CSV)

if "Latitude" in df_st.columns:
    df_st["Latitude"] = pd.to_numeric(df_st["Latitude"], errors="coerce")

if "Longitude" in df_st.columns:
    df_st["Longitude"] = pd.to_numeric(df_st["Longitude"], errors="coerce")

if "Capacity" in df_st.columns:
    df_st["Capacity"] = pd.to_numeric(df_st["Capacity"], errors="coerce").astype("Int64")


df_st = df_st.dropna(subset=["Latitude", "Longitude"]).copy()

if "StationId" in df_st.columns:
    df_st = df_st.drop_duplicates(subset=["StationId"]).copy()

print(f"dim_station: rows after cleaning = {len(df_st)}")


df_st.to_sql("dim_station", engine, schema="dbo", if_exists="replace", index=False)
print(f" Uploaded dbo.dim_station | rows={len(df_st)}")


df_tr = pd.read_csv(TRIPS_CSV)


for col in ["StartedAtOslo", "EndedAtOslo"]:
    if col in df_tr.columns:
        s = pd.to_datetime(df_tr[col], utc=True, errors="coerce")

        df_tr[col] = s.dt.tz_localize(None)


if "DurationSec" in df_tr.columns:
    df_tr["DurationSec"] = pd.to_numeric(df_tr["DurationSec"], errors="coerce").round().astype("Int64")

if "DurationMin" in df_tr.columns:
    df_tr["DurationMin"] = pd.to_numeric(df_tr["DurationMin"], errors="coerce")

for col in ["StartHourOslo", "DayOfWeek", "StartStationId", "EndStationId"]:
    if col in df_tr.columns:
        df_tr[col] = pd.to_numeric(df_tr[col], errors="coerce").astype("Int64")

if "IsWeekend" in df_tr.columns:
    df_tr["IsWeekend"] = df_tr["IsWeekend"].astype(bool)

print(f"fact_trip: rows after cleaning = {len(df_tr)}")

df_tr.to_sql("fact_trip", engine, schema="dbo", if_exists="replace", index=False)
print(f"Uploaded dbo.fact_trip   | rows={len(df_tr)}")


sample = pd.read_sql(
    "SELECT TOP 5 * FROM dbo.fact_trip ORDER BY StartedAtOslo;",
    engine
)

print("\nSample from dbo.fact_trip:")
print(sample)